# Notebook to develop the model for the project

In [2]:
# All our imports
import torch 
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib as plt
import numpy as np
from random import randint
#for all the plots to be inline
%matplotlib inline 


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


ModuleNotFoundError: No module named 'torch'

In [ ]:
'''Data Set manipulation'''

# build dataset of multiplications 
DATASET_SIZE = 100000
TESTSET_SIZE = 100
RANGE = 10 # from 0 to 100
LEARNING_RATE = .003


# building the training set
# dataset = np.zeros((DATASET_SIZE, 3), dtype=int)
dataset_inputs_int = np.random.randint(low=-RANGE, high=RANGE, size=(DATASET_SIZE, 2)) # 2 because of pairs
dataset_inputs = dataset_inputs_int.astype(np.float)
dataset_labels = np.zeros((DATASET_SIZE, 1), dtype=float)

# building the testing set
testset_inputs_int = np.random.randint(low=-RANGE, high=RANGE, size=(TESTSET_SIZE, 2)) # 30% for testing
testset_inputs = testset_inputs_int.astype(np.float)
testset_labels = np.zeros((DATASET_SIZE, 1), dtype=float)

#generate an array of the products
for index in range(DATASET_SIZE):
    # dataset[index,0] = randint(-10, 10)
    # dataset[index,1] = randint(-10, 10)
    # dataset[index,2] = dataset[index, 0] * dataset[index,1]
    dataset_labels[index, 0] = dataset_inputs[index, 0] * dataset_inputs[index, 1]


# print('dataset inputs\n', dataset_inputs)
# print('dataset labels\n', dataset_labels)

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # more analysis required to determine the specifics of the architecture
        self.n_input = 96*96 
        self.n_output = 8
        
#         imageInputLayer([28 28 1])
#         convolution2dLayer(3,8,'Padding','same')
#         batchNormalizationLayer
#         reluLayer
#         averagePooling2dLayer(2,'Stride',2)
#         convolution2dLayer(3,16,'Padding','same')
#         batchNormalizationLayer
#         reluLayer
#         averagePooling2dLayer(2,'Stride',2)
#         convolution2dLayer(3,32,'Padding','same')
#         batchNormalizationLayer
#         reluLayer
#         convolution2dLayer(3,32,'Padding','same')
#         batchNormalizationLayer
#         reluLayer
#         dropoutLayer(0.2)
#         fullyConnectedLayer(1)
#         regressionLayer
        
        

        
    def forward(self, x):
        #feedword pass through our network
        
        
        return x


    @staticmethod
    def load_checkpoint(new_model, filepath):
        checkpoint = torch.load(filepath)
        model = new_model.Network(checkpoint['input_size'], 
            checkpoint['output_size'],
            checkpoint['hidden_layers']
            )

        model.load_state_dict(checkpoint['state_dict'])

        return model
    
    def save(self):
        self.checkpoint = {
            'input_size': self.n_input, 
            'output_size': self.n_output,
            'hidden_layers': [each.out_features for each in model.hidden_layers],
            'state_dict': model.state_dict()
        }
        torch.save(self.checkpoint, 'checkpoint.pth')
    


In [ ]:
    
def validation(model, testloader, criterion):
    accuracy = 0
    test_loss = 0
    for images, labels in testloader:

        images = images.resize_(images.size()[0], 784)

        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ## Calculating the accuracy 
        # Model's output is log-softmax, take exponential to get the probabilities
        ps = torch.exp(output)
        # Class with highest probability is our predicted class, compare with true label
        equality = (labels.data == ps.max(1)[1])
        # Accuracy is number of correct predictions divided by all predictions, just take the mean
        accuracy += equality.type_as(torch.FloatTensor()).mean()

    return test_loss, accuracy


def train(model, trainloader, testloader, criterion, optimizer, epochs=5, print_every=40):
    
    steps = 0
    running_loss = 0
    for e in range(epochs):
        # Model in training mode, dropout is on
        model.train()
        for images, labels in trainloader:
            steps += 1
            
            # Flatten images into a 784 long vector
            images.resize_(images.size()[0], 784)
            
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

            if steps % print_every == 0:
                # Model in inference mode, dropout is off
                model.eval()
                
                # Turn off gradients for validation, will speed up inference
                with torch.no_grad():
                    test_loss, accuracy = validation(model, testloader, criterion)
                
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
                
                running_loss = 0
                
                # Make sure dropout and grads are on for training
                model.train()

In [ ]:
model = Network()